In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

In [2]:
import requests

url = "https://iselcouno.com/services/billing-rates"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the webpage
    html_content = response.text

    # Save the content to a file
    with open("iselco_webpage.html", "w", encoding="utf-8") as file:
        file.write(html_content)
    print("Webpage source code downloaded and saved as billing_rates_page.html")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Webpage source code downloaded and saved as billing_rates_page.html


In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

# Open the webpage
url = 'https://iselcouno.com/services/billing-rates'
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Use the XPath to locate all switcher items
switcher_items_xpath = '//*[@id="sheets-viewport"]/table/tbody/tr[2]/td/div/div[1]/table/tbody/tr/td'
switcher_items = driver.find_elements(By.XPATH, switcher_items_xpath)
print(switcher_items)

# Initialize a list to store DataFrames
dataframes = []

# Iterate over each switcher item
for index, switcher in enumerate(switcher_items, start=1):
    date_xpath = f'{switcher_items_xpath}[{index}]'
    date_element = driver.find_element(By.XPATH, date_xpath)
    date = date_element.text.strip()
    
    # Click on the switcher item to load the corresponding table
    date_element.click()
    time.sleep(2)  # Wait for the table to load after clicking
    
    # Locate the table after clicking
    try:
        table = driver.find_element(By.XPATH, "//table")
        # Read the table into a DataFrame
        df = pd.read_html(table.get_attribute('outerHTML'))[0]
        df['Date'] = date  # Add the date column
        dataframes.append(df)
    except Exception as e:
        print(f"Could not find a table for {date}: {e}")

# Combine all DataFrames into one
final_df = pd.concat(dataframes, ignore_index=True)

# Close the browser
driver.quit()

# Display the final DataFrame
print(final_df)

[]


ValueError: No objects to concatenate

### **Processing**